## Imports...

In [7]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math 
from collections import defaultdict

### First load all of the static image gestures. These are gestures that have no movement.

In [36]:
gesture_path = './gestures'
# If you recieve nothing from these lines make sure you have run nthe data_in scripts.
gesture_files = glob.glob("./gestures/*.csv")

def remove_prefix(text, prefix):
    if text.startswith(prefix):
        return text[len(prefix):]
    return text


def remove_suffix(text, suffix):
    if text.endswith(suffix):
        return text[: -1 * len(suffix)]
    return text


static_gestures = {}
# iterate through all of the gestures... Static pose is finding the median point of the dataframe.

# TODO: refit the transcription process to lead from correct pose point to pose point. instead of 0-20, 
# we need to do the math to the connecting limb to the center of the heart.
for indx, gesture_file in enumerate(gesture_files):
  
  df = pd.read_csv(gesture_file)
  
  gesture_file = remove_prefix(gesture_file, './gestures/')
  gesture_file = remove_suffix(gesture_file, '.csv')
  
  file_parts = gesture_file.split('_')
  gesture_name = file_parts[0].lower()
  gesture_part = f"{file_parts[2]}".lower()

  if df.empty:
      continue
  
  # convert string to time.
  df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S.%f').dt.time
  
  sdf = np.sort(df['time'].to_numpy(), axis=0)
  # Get the lower level bound median
  ts_median = sdf[math.floor(len(sdf) / 2 )]
  
  # Get the static point image/gesture...
  ndf = df[df['time'].isin([ts_median])]
  newGestureData = static_gestures.get(gesture_name, {})
  newGestureData[gesture_part] = ndf
  static_gestures[gesture_name] = newGestureData
  
print(static_gestures['one'])

{'left':                time  index     x_angle     y_angle     z_angle         x   
0   00:00:00.880836      0  259.006968   36.449836  323.041702  0.509624  \
1   00:00:00.880836      1  255.499880   45.422475  313.650641  0.471103   
2   00:00:00.880836      2  243.635986   46.992283  309.887212  0.447653   
3   00:00:00.880836      3  217.990059   40.411994  305.864388  0.437084   
4   00:00:00.880836      4   76.455848  354.713015  174.562671  0.446686   
5   00:00:00.880836      5  253.568500  358.149549    1.929186  0.448965   
6   00:00:00.880836      6  214.373893  324.677988   51.453873  0.463125   
7   00:00:00.880836      7  167.462176  325.969324  107.820987  0.470529   
8   00:00:00.880836      8   69.535184    3.055224  183.260608  0.463692   
9   00:00:00.880836      9  259.095298  351.191388    8.967983  0.476310   
10  00:00:00.880836     10  195.463377  298.617158   81.723193  0.492610   
11  00:00:00.880836     11  118.142727  328.840866  145.560960  0.500757   
12 

In [54]:
# Loop through our images to a static pose we know works.

all_gestures = static_gestures.keys()
  
# Proof of concept for the hand gesture "1"
static_base_gesture = static_gestures['one']['left']

for pose_name in all_gestures:
  print(pose_name)
  pose = static_gestures[pose_name]
  if pose.get('left', False) is False:
    continue
  pose_points = pose['left'].to_numpy()
  
  for idx, pose_point in enumerate(pose_points):
    
    x_angle = pose_point[2]
    y_angle = pose_point[3]
    z_angle = pose_point[4]
    
    base_x = static_base_gesture[idx][2]
    base_y = static_base_gesture[idx][3]
    base_z = static_base_gesture[idx][4]
    
    # Compute the error
    x_error = abs(base_x - x_angle)
    y_error = abs(base_y - y_angle)
    z_error = abs(base_z - z_angle)
    
  # This is the hard part... We need to fine tune the model for every static pose..
  # TODO: fine tune our error function to work for only the pose we want it to and no other poses.


born
[datetime.time(0, 0, 1, 785975) 0 270.5396323007981 30.822967295719295
 329.1759144990303 0.6903166174888611 0.6812841296195984
 0.0004395216528791] [datetime.time(0, 0, 1, 785975) 0 270.5396323007981 30.822967295719295
 329.1759144990303 0.6903166174888611 0.6812841296195984
 0.0004395216528791]
[datetime.time(0, 0, 1, 785975) 1 265.458669221022 62.133239672522826
 297.7923798503694 0.6477746367454529 0.658861517906189
 -0.0013414520071819] [datetime.time(0, 0, 1, 785975) 0 270.5396323007981 30.822967295719295
 329.1759144990303 0.6903166174888611 0.6812841296195984
 0.0004395216528791]
[datetime.time(0, 0, 1, 785975) 2 246.489233007658 84.30321503015182
 275.2266036738938 0.6138787865638733 0.6557608842849731
 -0.002690338762477] [datetime.time(0, 0, 1, 785975) 0 270.5396323007981 30.822967295719295
 329.1759144990303 0.6903166174888611 0.6812841296195984
 0.0004395216528791]
[datetime.time(0, 0, 1, 785975) 3 253.16792509296 67.97618704284476
 291.16538551982694 0.59084296226501